This notebook demonstrates preparing a ligand-protein system for OpenMM using the OpenFF Sage forcefield for the ligand and Amber99SB for the protein. 

This is a stripped down version of the "tookit_showcase" from OpenFF Toolkit.

In [ ]:
import openmm.app
import openmm
from openmmforcefields.generators import SMIRNOFFTemplateGenerator

from openff.toolkit.topology import Molecule

In [ ]:
# load both the Protein and Ligand structures
# OpenMM is used to correctly interpret the protein structure
# OpenFF is used for the ligand to prepare for the upcoming forcefield parametrisation

protein = openmm.app.PDBFile('3l9h_prepared.pdb')
ligand = Molecule('./chembl_1078774.sdf')

In [ ]:
# Load protein and water force field parameters
omm_forcefield = openmm.app.ForceField("amber99sb.xml", "tip3p.xml")
# The TemplateGenerator caters for the missing parameters caused by the ligand molecule
smirnoff = SMIRNOFFTemplateGenerator(forcefield="openff-2.0.0.offxml", molecules=[ligand])
omm_forcefield.registerTemplateGenerator(smirnoff.generator)

In [ ]:
# Combine structures and solvate
modeller = openmm.app.Modeller(protein.topology, protein.positions)
modeller.add(ligand.to_topology().to_openmm(), ligand.conformers[0])

# solvate it in 0.15 M NaCl solution
modeller.addSolvent(
    omm_forcefield,
    model="tip3p",
    padding=4.0 * openmm.unit.angstrom,  # 4.0 not enough, but works quickly
    ionicStrength=0.15 * openmm.unit.molar,
)

In [ ]:
system = omm_forcefield.createSystem(
    modeller.getTopology(),
    nonbondedMethod=openmm.app.PME,
    nonbondedCutoff=9 * openmm.unit.angstrom,
    constraints=openmm.app.HBonds,
    rigidWater=True,
)

In [ ]:
# Create a Simulation from our newly prepared System
# Construct and configure a Langevin integrator at 300 K with an appropriate friction constant and time-step
integrator = openmm.LangevinIntegrator(
    300 * openmm.unit.kelvin, 1 / openmm.unit.picosecond, 0.002 * openmm.unit.picoseconds
)

# Combine the topology, system, integrator and initial positions into a simulation
simulation = openmm.app.Simulation(modeller.getTopology(), system, integrator)
simulation.context.setPositions(modeller.getPositions())